## 1. 문제정의 - 은행 대출 승인 여부

In [21]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import numpy as np

## 2. 데이터 수집

In [22]:
x_train=pd.read_csv("bank_x_train.csv")
y_train=pd.read_csv("bank_y_train.csv")
x_test=pd.read_csv("bank_x_test.csv")
y_test=pd.read_csv("bank_y_test.csv")

del x_train['Unnamed: 0']
del x_test['Unnamed: 0']
del y_train['Unnamed: 0']
del y_test['Unnamed: 0']
x_train=x_train.drop(['Loan_ID'],axis=1)
x_test=x_test.drop(['Loan_ID'],axis=1)

### 원핫인코딩

In [23]:
transformer=make_column_transformer(
    (OneHotEncoder(),
    ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']),
    remainder='passthrough')

In [24]:
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Gender', 'Married', 'Dependents',
                                  'Education', 'Self_Employed',
                                  'Property_Area'])])

In [25]:
x_train=transformer.transform(x_train)
x_test=transformer.transform(x_test)
x_train

array([[  0.,   1.,   1., ..., 125., 360.,   1.],
       [  0.,   1.,   0., ..., 123., 480.,   1.],
       [  0.,   1.,   0., ...,  56., 180.,   1.],
       ...,
       [  0.,   1.,   0., ..., 211., 360.,   1.],
       [  0.,   1.,   1., ..., 144., 360.,   1.],
       [  0.,   1.,   0., ..., 255., 360.,   1.]])

In [42]:
labels=['비승인','승인']
le=LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train)
y_test=le.transform(y_test)

In [29]:
#Voting : soft
model = VotingClassifier(estimators=[('lr',LogisticRegression()),
                                   ('kn',KNeighborsClassifier()),
                                   ('dt',DecisionTreeClassifier())])
model.fit(x_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('kn', KNeighborsClassifier()),
                             ('dt', DecisionTreeClassifier())])

In [30]:
model.score(x_train,y_train)

0.8932291666666666

In [31]:
model.score(x_test,y_test)

0.7291666666666666

#### 새로운 값으로 예측

In [39]:
x_test=[['Male',"Yes","0","Graduate","No",14583,0,436,360,1,'Semiurban']]

x_test=pd.DataFrame(x_test,columns=['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 
        'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 
        'Loan_Amount_Term', 'Credit_History', 'Property_Area'])
x_test=transformer.transform(x_test)

In [40]:
y_predict=model.predict(x_test)

In [43]:
labels[y_predict[0]]

'승인'